In [2]:
import json
from pathlib import Path

import hnswlib as hb
import numpy as np
import pandas as pd
import requests
import srsly
from datasets import (
    ClassLabel,
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value,
    load_dataset,
    load_from_disk,
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

In [8]:
df = (
    load_from_disk("/home/pl487/anchoral/data/prepared/amazoncat-13k-agri_bert-tiny")["train"]
    .select_columns(["uid", "text", "labels"])
    .to_pandas()
)

In [9]:
df["labels"].value_counts(normalize=True)

labels
0    0.999114
1    0.000886
Name: proportion, dtype: float64

In [64]:
df = (
    load_from_disk("/home/pl487/anchoral/data/processed/amazoncat-13k")["test"]
    .select_columns(["uid", "text", "target_ind"])
    .to_pandas()
)

In [42]:
with open(
    "/home/pl487/anchoral/data/processed/amazoncat-13k/categories.txt", mode="r", encoding="latin-1"
) as fl:
    cats = fl.read().split("\n")
cats = pd.DataFrame(cats, columns=["name"])

In [65]:
dd = df[["uid", "target_ind"]].explode("target_ind")

In [66]:
dd = dd.groupby("target_ind")["uid"].nunique().to_frame("counts")

In [67]:
dd["prop"] = dd["counts"] / dd["counts"].sum()

In [68]:
dd = dd.join(cats)

In [70]:
dd.query("target_ind == 246")

,counts,prop,name
target_ind,,,
246,285,0.000182,agricultural sciences


In [69]:
dd.query("1000 < counts < 2000")

,counts,prop,name
target_ind,,,
38,1240,0.000792,19th century
39,1952,0.001247,20th century
118,1322,0.000844,action & adventure
119,1680,0.001073,action & toy figures
442,1620,0.001035,animals
...,...,...,...
12993,1017,0.000650,web development & design
13214,1316,0.000841,women's studies
13258,1128,0.000721,world literature


In [49]:
mask = df["target_ind"].map(lambda ex: 246 in ex)

In [56]:
co = df.loc[mask, "target_ind"].explode().value_counts().to_frame()

In [63]:
co.join(cats).sort_values("count", ascending=False).head(50)

,count,name
target_ind,,
246,1051,agricultural sciences
1471,1032,books
10368,1032,science & math
12630,382,used & rental textbooks
8108,382,new
439,336,animal husbandry
1280,314,biological sciences
3125,296,crafts
5904,296,hobbies & home


In [ ]:
index_path = "/home/pl487/anchoral/data/processed/eurlex-57k/all-MiniLM-L12-v2_cosine.bin"
meta_path = "/home/pl487/anchoral/data/processed/eurlex-57k/all-MiniLM-L12-v2_cosine.json"

In [ ]:
meta = srsly.read_json(meta_path)
index = hb.Index(space=meta["metric"], dim=meta["dim"])
index.load_index(str(index_path))

In [ ]:
index.mark_deleted(0)

In [ ]:
np.stack(index.get_items([0])).shape

In [ ]:
sorted(index.get_ids_list())

In [ ]:
ds_dict = load_from_disk("/home/pl487/anchoral/data/processed/wiki_toxic")

In [ ]:
ds_dict["train"].to_pandas()["input_ids"].map(len).describe()

In [ ]:
df = ds_dict["train"].to_pandas()

In [ ]:
df["labels"].value_counts()

---
Pubmed

In [ ]:
d = load_from_disk("/home/pl487/anchoral/data/processed/pubmed-200k-rct")

In [ ]:
d

In [ ]:
df = d["train"].to_pandas()

In [ ]:
df.loc[df["uid"] == 40384]

In [ ]:
df["recitals"].str.len().describe()

In [ ]:
df["text"].str.len().describe()

In [ ]:
# ds_dict = load_dataset("pietrolesci/eurlex_indexeda", cache_dir="ds_cache")
# ds_dict.save_to_disk("../data/processed/eurlex")

---
Pubmed

In [10]:
df = (
    load_from_disk("/home/pl487/anchoral/data/processed/pubmed-200k-rct/")["train"]
    .select_columns(["uid", "text", "labels"])
    .to_pandas()
)

In [12]:
df["labels"].value_counts(normalize=True)

labels
4    0.346437
2    0.326687
1    0.153587
0    0.088925
3    0.084364
Name: proportion, dtype: float64

In [ ]:
d = load_from_disk("/home/pl487/anchoral/data/prepared/pubmed-200k-rct_bert-tiny")

In [ ]:
d

In [ ]:
ds_dict = load_from_disk("/home/pl487/anchoral/data/processed/amazoncat-13k/").select_columns(
    ["uid", "target_ind", "text"]
)

In [ ]:
train_df = ds_dict["train"].to_pandas()
test_df = ds_dict["test"].to_pandas()

In [ ]:
a = train_df["target_ind"].explode().value_counts(True).to_frame("f")

In [ ]:
a.loc[(a["f"] > 0.01) & (a["f"] < 0.02)].sort_values("f", ascending=False).join(cats)

In [ ]:
ids = cats.loc[cats[0].str.contains("religion")].index.tolist()
cats.loc[cats[0].str.contains("religion")]

In [ ]:
ids

In [ ]:
mask = train_df["target_ind"].map(lambda ex: any(i in ex for i in ids))

In [ ]:
train_df.loc[mask]

In [ ]:
mask.sum() / len(train_df)

In [ ]:
fiction = 8%
movies = 7%
social science = 4%
religion = 2%

In [ ]:
train_df.loc[train_df["target_ind"].map(lambda ex: 7893 in ex), "text"].to_dict()

In [ ]:
# df.loc[df["target_ind"].map(lambda ex: 13199 in ex)]
df.loc[df["target_ind"].map(lambda ex: 12583 in ex)].iloc[2].to_dict()

In [ ]:
30068 / len(df)

In [ ]:
MODELS = ["all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-MiniLM-L12-v2"]

In [ ]:
models = [f"embedding_{i}" for i in MODELS]
df = load_dataset("pietrolesci/eurlex_indexed")["train"].to_pandas()
df = df[["uid"] + models]

In [ ]:
metric: str = "cosine"
ef_construction: int = 200
ef: int = 200
M: int = 64
num_threads: int = 5

path = Path("../data/processed/eurlex/hnswlib_indices")
path.mkdir(exist_ok=True, parents=True)

for idx, model in enumerate(tqdm(models)):

    path.mkdir(exist_ok=True, parents=True)

    embeddings = np.stack(df[model].values)

    index = hb.Index(space=metric, dim=embeddings.shape[1])
    index.set_ef(ef)
    index.init_index(
        max_elements=embeddings.shape[0],
        M=M,
        ef_construction=ef_construction,
        random_seed=42,
    )

    index.add_items(embeddings, df["uid"].tolist(), num_threads=num_threads)
    index.save_index(str(path / f"index_{metric}_{MODELS[idx]}.bin"))

    srsly.write_yaml(
        path / f"metadata_{metric}_{MODELS[idx]}.yaml",
        {
            "metric": metric,
            "ef_construction": ef_construction,
            "ef": ef,
            "M": M,
            "num_threads": num_threads,
        },
    )

In [ ]:
data = ds_dict.map(
    lambda ex: {"labels": [int("3191" in l) for l in ex["eurovoc_concepts"]]},
    batched=True,
)

In [ ]:
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_freq = (
    train_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_train")
)
test_freq = (
    test_df[["celex_id", "eurovoc_concepts"]]
    .explode("eurovoc_concepts")
    .groupby("eurovoc_concepts")["celex_id"]
    .nunique()
    .to_frame("count_test")
)

In [ ]:
desc = pd.DataFrame(srsly.read_jsonl("../data/raw/eurlex-57k/eurovoc_concepts.jsonl")).set_index(
    "id"
)

In [ ]:
f = train_freq.join(test_freq).dropna()

In [ ]:
f["sum"] = f.sum(1)

In [ ]:
f = f.sort_values("sum", ascending=False)

In [ ]:
f = f.join(desc)

In [ ]:
f.loc[(f["sum"] < 3500) & (f["sum"] > 1000)].sort_values(
    ["count_test", "count_train"], ascending=[False, True]
)

In [ ]:
# make `health control` the target label
data = ds_dict.map(
    lambda ex: {"labels": [int("192" in l) for l in ex["eurovoc_concepts"]]},
    batched=True,
)
train_df = data["train"].to_pandas()
test_df = data["test"].to_pandas()

In [ ]:
train_df["labels"].value_counts(True), test_df["labels"].value_counts(True)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
log_reg = LogisticRegression()

In [ ]:
train_df["document_type"].value_counts()

In [ ]:
# X = tfidf.fit_transform(train_df["text"].values)
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values
# y = train_df["document_type"] == "Directive"

# X_test = tfidf.transform(test_df["text"].values)
X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values
# y_test = test_df["document_type"] == "Directive"

In [ ]:
log_reg.fit(X, y)

In [ ]:
print(classification_report(y_test, log_reg.predict(X_test)))

In [ ]:
print(classification_report(y, log_reg.predict(X)))

In [ ]:
ds_dict

In [ ]:
1609 / len(train_df)

In [ ]:
train_df["document_type"].value_counts(True)

In [ ]:
train_df.loc[train_df["eurovoc_concepts"].map(lambda ex: "2173" in ex)]

In [ ]:
desc

In [ ]:
# get eurlex from http://nlp.cs.aueb.gr/software_and_datasets/EURLEX57K/ and unzip the folder into eurlex
path = Path("../eurlex/")
data = {}
for split in path.iterdir():
    data[split.name] = []
    for file in tqdm(list(split.iterdir())):
        data[split.name].append(srsly.read_json(file))

In [ ]:
cats = [
    "Wireless_v1_00",
    "Watches_v1_00",
    "Video_Games_v1_00",
    "Video_DVD_v1_00",
    "Video_v1_00",
    "Toys_v1_00",
    "Tools_v1_00",
    "Sports_v1_00",
    "Software_v1_00",
    "Shoes_v1_00",
    "Pet_Products_v1_00",
    "Personal_Care_Appliances_v1_00",
    "PC_v1_00",
    "Outdoors_v1_00",
    "Office_Products_v1_00",
    "Musical_Instruments_v1_00",
    "Music_v1_00",
    "Mobile_Electronics_v1_00",
    "Mobile_Apps_v1_00",
    "Major_Appliances_v1_00",
    "Luggage_v1_00",
    "Lawn_and_Garden_v1_00",
    "Kitchen_v1_00",
    "Jewelry_v1_00",
    "Home_Improvement_v1_00",
    "Home_Entertainment_v1_00",
    "Home_v1_00",
    "Health_Personal_Care_v1_00",
    "Grocery_v1_00",
    "Gift_Card_v1_00",
    "Furniture_v1_00",
    "Electronics_v1_00",
    "Digital_Video_Games_v1_00",
    "Digital_Video_Download_v1_00",
    "Digital_Software_v1_00",
    "Digital_Music_Purchase_v1_00",
    "Digital_Ebook_Purchase_v1_00",
    "Camera_v1_00",
    "Beauty_v1_00",
    "Baby_v1_00",
    "Automotive_v1_00",
    "Apparel_v1_00",
    "Digital_Ebook_Purchase_v1_01",
    "Books_v1_00",
    "Books_v1_01",
    "Books_v1_02",
]

In [ ]:
ds_dict = load_dataset("pietrolesci/pubmed-rct-200k_indexed")
ds_dict = ds_dict.select_columns(["uid", "labels", "text", "embedding_all-mpnet-base-v2"])

train_df = ds_dict["train"].to_pandas()
test_df = ds_dict["test"].to_pandas()

In [ ]:
X = np.stack(train_df["embedding_all-mpnet-base-v2"].values)
y = train_df["labels"].values == 0

X_test = np.stack(test_df["embedding_all-mpnet-base-v2"].values)
y_test = test_df["labels"].values == 0

In [ ]:
train_df["labels"].value_counts(True)

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X, y)

In [ ]:
print(classification_report(y_test, log_reg.predict(X_test)))

In [ ]:
print(classification_report(y, log_reg.predict(X)))

In [ ]:
len(train_df)

In [ ]:
d = load_dataset("pietrolesci/pubmed-rct-200k_indexed")

In [ ]:
a